In [ ]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

In [ ]:
import re

def preprocess_input(input):
    #regexp = r'[\\\/\.,;:()]+\w*'
    #regexp2 = r'\b(?<![\\\/\.\,\\:\;\(\)\+\-])\w+\b'
    #regexp3 = r'\b(?<![\\\/])\w+\.*\w*\b'

    # Remove unicode and webpages
    regexp4 = r'\b(?<![\\\.])\w+(?!\.\w+)\b'

    # Regex for removing words that are preceded by '\'
    unicode_regexp = r'\b(?<![\\])\w+\b'

    # Regex for removing webpages
    webpage_regexp = r'\b(?<![\.])\w+(?!\.\w+)\b'

    # Remove html tags
    result = BeautifulSoup(input, 'lxml').get_text()
    


    # Save words that are not preceded by a '\' and webpages
    result = re.findall(regexp4, result)
    
            # TRYING TO COMBINE MULTIPLE REGEX, WORK IN PROGRESS
            #patterns = [unicode_regexp, webpage_regexp]
            #pattern = "|".join(patterns)  # pattern1|pattern2|pattern3|...
            #re.findall(pattern, result)

    # Remove excessive whitespaces
    result = " ".join(result)


    #result = " ".join(result.split())
   
    return result

In [ ]:
from os.path import exists

if(not exists('preprocessed_articles.csv')):  
    df_articles = pd.read_csv('sävsjö_articles.csv')
    df_articles = df_articles.dropna()
    # columns: ['ID', 'Title', 'Text']
    # iterate over 'Title' and 'Text' columns and preprocess the texts
    for col in df_articles.columns[1:]:
        df_articles[col] = df_articles[col].astype(str)
        df_articles[col] = df_articles[col].apply(lambda x: preprocess_input(x))
    
else:
    df_articles = pd.read_csv('preprocessed_articles.csv')

In [ ]:
df_articles.head()

In [ ]:
#from spacy.pipeline.ner import EntityRecognizer
import spacy
from os.path import exists

if(not exists('preprocessed_articles.csv')):   

    nlp = spacy.load("sv_core_news_sm")
    df_articles['Entities'] = '0'
    df_articles['Labels'] = '0'

    for index, row in df_articles.iterrows():
        doc_text = nlp(row['Text'])
        doc_title = nlp(row['Title'])
    
        doc_ents = doc_text.ents + doc_title.ents

        # Remove duplicate entities from the list
        unique_ents = list({keyword.__repr__(): keyword for keyword in doc_ents}.values())

        # Remove entities that have label 'TME' because the words are mostly redundant
        unique_ents = [ent for ent in unique_ents if not ent.label_ in ['TME']]        
        
        entities = [str(x) for x in unique_ents]
        labels = [str(labels.label_) for labels in unique_ents]
        
        df_articles.loc[index,['Entities', 'Labels']] = f'{entities}', f'{labels}'
        if index > 10:
            break
    df_articles.to_csv('preprocessed_articles.csv')

df_articles.head(10)


# remove empty rows and convert to lists.

In [ ]:
df_test = df_articles[["Labels","Entities"]]

for ind, row in df_test.iterrows():
    if row["Labels"] == "[]":
        df_test = df_test.drop(index=ind)

df_test[["Labels","Entities"]].apply(lambda x : str(x).split(','))